# Calculate crossing reactions

This function calculates substitution rules of s,t,u for crossing reactions

The procedure is explained in the book in section 3.2 and in the appendix A.2. 


In [10]:
import sympy as sy
import math

In [11]:
p1,p2,p3,p4 = sy.symbols("p_1 p_2 p_3 p_4",real=True)
S,T,U = sy.symbols("S T U",real=True)
s,t,u = sy.symbols("s t u",real=True)
m,M = sy.symbols("m M",positive=True)

In [12]:
def crossing(particles, order, Tfisq_base = None):
    """
    
    This function calculates substitution rule of s,t,u for crossing reactions

    Parameters
    ----------
    particles (list of strings) : gives names of particles of base reaction. 
                                e.g. particles = ['e-','e+','mu-','mu+']
    order (list of integers) : gives new order of particles after crossing
                                base order = [1,2,3,4]
                                after crossing: order = [1,4,2,3]
    Tfisq_base (str, optional) : One can give the string expression of the 
                                 spin averaged amplitude squared of the base reaction using s,t,u.  
                                e.g. Tfisq_base = "(t**2 + u**2)/s**2"  (pre-factots omitted here)
    Returns
    -------- 
    None

    Prints
    ------
    - base reaction
    - crossed reaction
    - substitution rules for s,t,u
    - Amplitude sign : +1 or -1, relative sign between the amplitude-squared of both reactions 
    - Tfisq of crossed reaction (if Tfisq_base is given). Amplitude sign is applied. 

    Several examples of usage are given below following the discusion in the book. 

    """

    # make particle dict
    dpar = {'gam':0, 
        'nue':1, 'nuebar':-1, 
        'numu':2, 'numubar':-2,
        'nutau':3, 'nutaubar':-3,
        'e-':4, 'e+':-4,
        'mu-':5, 'mu+':-5,
        'tau-':6, 'tau+':-6,
        'u':7, 'ubar':-7,
        'd':8, 'dbar':-8,
        'c':9, 'cbar':-9,
        's':10, 'sbar':-10,
        't':11, 'tbar':-11,
        'b':12, 'bbar':-12,
        'W+':13, 'W-':-13,
        'Z':14, 'H':15,
        'A':20, 'Abar':-20,
        'B':21, 'Bbar':-21,
        'C':22, 'Cbar':-22,
        'D':23, 'Dar':-23}       
    # make revesed dict {4:'e-',....} and add -14,-15 for Z, H
    dpar_flipped = {value: key for key, value in dpar.items()}
    dpar_flipped[-14] = 'Z'
    dpar_flipped[-15] = 'H'
    # Mandelstam variables 
    dsubs = {(p1+p2)**2: S, (p3+p4)**2:S, (-p1-p2)**2: S, (-p3-p4)**2:S,
         (p1-p3)**2: T, (p2-p4)**2: T,(-p1+p3)**2: T, (-p2+p4)**2: T,
         (p1-p4)**2: U, (p2-p3)**2: U, (-p1+p4)**2: U, (-p2+p3)**2: U
    }
    # momentum assignment
    lmom = [p1,p2,p3,p4]
    lpar = []
    for name in particles:
        lpar.append(dpar[name])
    # 
    lparc = []
    lmomc = []
    Asign = 1
    # calculate crossing
    for i in range(1,5):
        p = order[i-1]
        # calculate sign change for momentum
        sign = int(math.copysign(1,2.5-i)*math.copysign(1,2.5-p))
        # calculate overall sign for Amplitude
        if abs(lpar[p-1]) >= 1 and abs(lpar[p-1]) <= 12:
            Asign = Asign*sign
            #print("i: {}, p: {}, lpar[p-1]: {}, sign: {}, Asign: {}".format(
            #    i,p,lpar[p-1],sign,Asign))
        pc = lpar[p-1]*sign
        mc = lmom[p-1]*sign
        lparc.append(pc)
        lmomc.append(mc)
    # Calculate Mandelstam
    sc = (lmomc[0]+lmomc[1])**2
    tc = (lmomc[0]-lmomc[2])**2
    uc = (lmomc[0]-lmomc[3])**2
        
    # get crossed particle names
    particlesc = []
    for num in lparc:
        particlesc.append(dpar_flipped[num])
    # printout    
    print("Base reaction: {}  {} --> {}  {}".format(*particles))
    print("Crossed reaction: {}  {} --> {}  {}".format(*particlesc))
    print("Exchange Mandelstam variables between crossed and base (capital letters) reaction:")
    print("s <-> ",sc.subs(dsubs))
    print("t <-> ",tc.subs(dsubs))
    print("u <-> ",uc.subs(dsubs))

    print("Amplitude sign: ", Asign)
    if not Tfisq_base == None:
        Tfisq_base = Tfisq_base.subs(s,S).subs(t,T).subs(u,U)
        Tfisq = Tfisq_base
        Tfisq = Tfisq.subs(sc.subs(dsubs),s)
        Tfisq = Tfisq.subs(tc.subs(dsubs),t)
        Tfisq = Tfisq.subs(uc.subs(dsubs),u)
        Tfisq = Tfisq*Asign
        display("Tfisq_base: ",Tfisq_base)
        display("Tfisq_crossed: ",Tfisq)

# Examples

#### Base reaction:  $e^- ~ e^+ \to \mu^-  \mu^+$ 
#### Crossed reaction:  $e^- ~ \mu^- \to e^-  \mu^-$

In [13]:
# Base reaction: e-  e+ --> mu-  mu+
# Crossed reaction: e-  mu- --> e-  mu-
# see. Book. eq. 3.62 and 3.63
particles = ['e-','e+','mu-','mu+']
order = [1,4,2,3]
Tfisq_base = (t**2 + u**2)/s**2
crossing(particles, order, Tfisq_base = Tfisq_base)

Base reaction: e-  e+ --> mu-  mu+
Crossed reaction: e-  mu- --> e-  mu-
Exchange Mandelstam variables between crossed and base (capital letters) reaction:
s <->  U
t <->  S
u <->  T
Amplitude sign:  1


'Tfisq_base: '

(T**2 + U**2)/S**2

'Tfisq_crossed: '

(s**2 + u**2)/t**2

In [14]:
# Base reaction: e-  e+ --> e-  e+
# Crossed reaction: e-  mu- --> e-  mu-
# see. Book. eq. 3.77ff
# Calculate with masses
# the amplitude has 3 terms, calculate them seperately in turn
particles = ['e-','e+','e-','e+']
order = [1,4,3,2]
Tfisq_base = (S**2+U**2+8*m**2*(T-m**2))/T**2 
Tfisq_base = (T**2+U**2+8*m**2*(S-m**2))/S**2
Tfisq_base = 2*(U-2*m**2)*(u-6*m**2)/ (t*s)
crossing(particles, order, Tfisq_base = Tfisq_base)

Base reaction: e-  e+ --> e-  e+
Crossed reaction: e-  e- --> e-  e-
Exchange Mandelstam variables between crossed and base (capital letters) reaction:
s <->  U
t <->  T
u <->  S
Amplitude sign:  1


'Tfisq_base: '

(U - 6*m**2)*(2*U - 4*m**2)/(S*T)

'Tfisq_crossed: '

(-6*m**2 + s)*(-4*m**2 + 2*s)/(t*u)

#### Base reaction:  $e^- ~ e^+ \to \mu^-  \mu^+$ <br>
#### Crossed reaction:  $e^- ~ \mu^+ \to e^-  \mu^+$

In [15]:
# Base reaction: e-  e+ --> mu-  mu+
# Crossed reaction: e-  mu+ --> e-  mu+
# see. Book. eq. 3.62 and 3.63
particles = ['e-','e+','mu-','mu+']
Tfisq_base = (t**2 + u**2)/s**2
crossing(particles, [1,3,2,4], Tfisq_base = Tfisq_base)

Base reaction: e-  e+ --> mu-  mu+
Crossed reaction: e-  mu+ --> e-  mu+
Exchange Mandelstam variables between crossed and base (capital letters) reaction:
s <->  T
t <->  S
u <->  U
Amplitude sign:  1


'Tfisq_base: '

(T**2 + U**2)/S**2

'Tfisq_crossed: '

(s**2 + u**2)/t**2

#### Base reaction:  $e^- ~ e^+ \to \gamma  \gamma$ <br>
#### Crossed reaction:  $e^- ~ \gamma \to e^- \gamma$

In [16]:
# Base reaction: e-  e+ --> gam  gam
# Crossed:  gam e- --> gam e-  (1. option)
# see book eq. 3.87 and 3.94 ff.
# as explained in the book Tfi gets a minus sign, 
# because only one fermion is exchanged between incoming and outgoing. 
particles = ['e-','e+','gam','gam']
order = [3,1,4,2] # 1st option for ordering
Tfisq_base = (u/t + t/u)
crossing(particles, [1,3,2,4], Tfisq_base = Tfisq_base)

Base reaction: e-  e+ --> gam  gam
Crossed reaction: e-  gam --> e-  gam
Exchange Mandelstam variables between crossed and base (capital letters) reaction:
s <->  T
t <->  S
u <->  U
Amplitude sign:  -1


'Tfisq_base: '

T/U + U/T

'Tfisq_crossed: '

-s/u - u/s

In [17]:
# Base reaction: e-  e+ --> gam  gam
# Crossed:  gam e- --> gam e-  (2. option)
# see book eq. 3.87 and 3.94 ff.
# as explained in the book Tfi gets a minus sign, 
# because only one fermion is exchanged between incoming and outgoing.
# Note that we get the same result for both ordering options.  
particles = ['e-','e+','gam','gam']
order = [1,4,2,3] # 2nd option for ordering
Tfisq_base = (u/t + t/u)
crossing(particles, order, Tfisq_base = Tfisq_base)

Base reaction: e-  e+ --> gam  gam
Crossed reaction: e-  gam --> e-  gam
Exchange Mandelstam variables between crossed and base (capital letters) reaction:
s <->  U
t <->  S
u <->  T
Amplitude sign:  -1


'Tfisq_base: '

T/U + U/T

'Tfisq_crossed: '

-s/u - u/s


#### Base reaction:  $\nu_e ~ e^- \to \nu_e ~ e^-$ <br>
#### Crossed reaction:  $\bar\nu_e ~ e^- \to \bar\nu_e ~ e^-$

In [18]:
# Base reaction: nu_e-  e- --> nu_e  e- 
# Crossed:  nubar_e e-  --> gam nubar_e e- 
# see book eq. 6.9ff
# only the charged current reaction is used  
particles = ['nue','e-','nue','e-']
order = [3,2,1,4] 
Tfisq_base = S
crossing(particles, order, Tfisq_base = Tfisq_base)

Base reaction: nue  e- --> nue  e-
Crossed reaction: nuebar  e- --> nuebar  e-
Exchange Mandelstam variables between crossed and base (capital letters) reaction:
s <->  U
t <->  T
u <->  S
Amplitude sign:  1


'Tfisq_base: '

S

'Tfisq_crossed: '

u